In [1]:
import pandas as pd
import re
from nltk.corpus import stopwords
import io
from sklearn.metrics import precision_recall_fscore_support
from sklearn.cluster import MiniBatchKMeans
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
Train = pd.read_csv("Train.csv")
stop_words = stopwords.words("english")
f = io.open("words.txt", mode="r", encoding="utf-8")
stop_words = set(stopwords.words('english')) 
wordlist = f.read().split("\n")
wordlist = [word for word in wordlist if word not in stop_words]
f.close()

In [2]:
Train.Review = Train['Review'].map(lambda a: re.sub(r'[^\w\s]','',a).lower())
Train.Review = Train['Review'].map(lambda a: ' '.join([word for word in a.split() if word not in stop_words]))

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(vocabulary = wordlist, encoding="ISO-8859-1", strip_accents=ascii, analyzer='word')
bagofwords_train = vectorizer.fit_transform(Train['Review'])

In [4]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
tf_idf_train = tfidf_transformer.fit_transform(bagofwords_train)

In [5]:
kmeans = MiniBatchKMeans(n_clusters=5, random_state=0, batch_size=500, max_iter=5000).fit(tf_idf_train)
labels = pd.DataFrame(kmeans.labels_, columns = ["Cluster"])

Model for cluster 0

In [6]:
Train = pd.read_csv("Train.csv")
x_Train = tf_idf_train[labels.index[labels['Cluster'] == 0].tolist(), :]
y_Train = Train.iloc[labels.index[labels['Cluster'] == 0].tolist(), :]['Class']
Cluster0_model = LogisticRegression().fit(x_Train, y_Train)

c:\users\aditw\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Model for cluster 1

In [7]:
x_Train = tf_idf_train[labels.index[labels['Cluster'] == 1].tolist(), :]
y_Train = Train.iloc[labels.index[labels['Cluster'] == 1].tolist(), :]['Class']
Cluster1_model = LogisticRegression().fit(x_Train, y_Train)

Model for Cluster 2

In [9]:
x_Train = tf_idf_train[labels.index[labels['Cluster'] == 2].tolist(), :]
y_Train = Train.iloc[labels.index[labels['Cluster'] == 2].tolist(), :]['Class']
Cluster2_model = LogisticRegression().fit(x_Train, y_Train)

Model for Cluster 3

In [8]:
x_Train = tf_idf_train[labels.index[labels['Cluster'] == 3].tolist(), :]
y_Train = Train.iloc[labels.index[labels['Cluster'] == 3].tolist(), :]['Class']
Cluster3_model = LogisticRegression().fit(x_Train, y_Train)

Model for Cluster 4

In [10]:
x_Train = tf_idf_train[labels.index[labels['Cluster'] == 4].tolist(), :]
y_Train = Train.iloc[labels.index[labels['Cluster'] == 4].tolist(), :]['Class']
Cluster4_model = LogisticRegression().fit(x_Train, y_Train)

Model for Cluster 5

In [11]:
# x_Train = tf_idf_train[labels.index[labels['Cluster'] == 5].tolist(), :]
# y_Train = Train.iloc[labels.index[labels['Cluster'] == 5].tolist(), :]['Class']
# Cluster5_model = LogisticRegression().fit(x_Train, y_Train)

Model for Cluster 6

In [12]:
# x_Train = tf_idf_train[labels.index[labels['Cluster'] == 6].tolist(), :]
# y_Train = Train.iloc[labels.index[labels['Cluster'] == 6].tolist(), :]['Class']
# Cluster6_model = LogisticRegression().fit(x_Train, y_Train)

In [13]:
Test = pd.read_csv("imdb_ts.csv")
y_true = Test['Class']
Test.Review = Test['Review'].map(lambda a: re.sub(r'[^\w\s]','',a).lower())
Test.Review = Test['Review'].map(lambda a: ' '.join([word for word in a.split() if word not in stop_words]))
vectorizer = CountVectorizer(vocabulary = wordlist, encoding="ISO-8859-1", strip_accents=ascii, analyzer='word')
bagofwords_test = vectorizer.fit_transform(Test['Review'])
tfidf_transformer = TfidfTransformer()
tf_idf_test = tfidf_transformer.fit_transform(bagofwords_test)
cluster_predict = kmeans.predict(tf_idf_test)

In [15]:
correct = 0
tp = 0
tn = 0
fp = 0
fn = 0

i = 0
for x in np.nditer(cluster_predict):
    if(x == 0):
        prediction = Cluster0_model.predict(tf_idf_test[i])[0]
    elif(x == 1):
        prediction = Cluster1_model.predict(tf_idf_test[i])[0]
    elif(x == 2):
        prediction = Cluster2_model.predict(tf_idf_test[i])[0]
    elif(x == 3):
        prediction = Cluster3_model.predict(tf_idf_test[i])[0]
    elif(x == 4):
        prediction = Cluster4_model.predict(tf_idf_test[i])[0]
#     elif(x == 5):
#         prediction = Cluster5_model.predict(tf_idf_test[i])[0]
#     elif(x == 6):
#         prediction = Cluster6_model.predict(tf_idf_test[i])[0]
    
    if(y_true[i] == 1 and prediction == 1):
            tp+=1
            correct+=1
    elif(y_true[i] == 0 and prediction == 0):
            tn+=1
            correct+=1
    elif(y_true[i] == 1 and prediction == 0):
            fn+=1
    elif(y_true[i] == 0 and prediction == 1):
            fp+=1
    i = i+1
        
print(correct/25000)
print("Recall ", tp/(tp+fn))
print("Precision ", tp/(tp+fp))


0.85852
Recall  0.86592
Precision  0.8532912889239259


In [16]:
print(" ", "1   ", "0")
print("1", tp, fp)
print("0", fn, tn)

  1    0
1 10824 1861
0 1676 10639
